In [ ]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models

warnings.filterwarnings('ignore')

from dataset import *
from transform import *

In [3]:
IMG_SIZE = 224
data_path = r"C:\Users\Admin\Desktop\FINAL-IMAGE-PROCESSING\deep-learning-for-images\data\gender-classification"

train_transform = gender_transforms(is_train=True)
val_transform = gender_transforms(is_train=False)
test_transform = gender_transforms(is_train=False)

# Datasets
train_dataset = GenderDataset(
    root_dir=os.path.join(data_path, 'Train'),
    transform=train_transform
)
val_dataset = GenderDataset(
    root_dir=os.path.join(data_path, 'Val'),
    transform=val_transform
)
test_dataset = GenderDataset(
    root_dir=os.path.join(data_path),
    transform=test_transform,
    is_test=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True, 
    num_workers=0,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=0,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=0,
    pin_memory=True
)

Train dataset size: 8000
Validation dataset size: 1000
Test dataset size: 1000


In [4]:
images, labels = next(iter(train_loader))
print(images.shape)  # (batch_size, 3, H, W)
print(labels.shape)  # (batch_size,)
print(images[0].shape)
print(labels[0].shape)

torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([3, 224, 224])
torch.Size([])


In [5]:
# Load the pretrained ResNet-18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer for binary classification (male/female)
model.fc = nn.Linear(model.fc.in_features, 2)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# TRAIN IN ONE EPOCH FOR TESTING
for epoch in range(1):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


In [7]:
# EVALUATE
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total:.2f}%")


Accuracy on validation set: 46.30%
